In [2]:
trainpath="Alzheimer_s Dataset\train"
testpath="Alzheimer_s Dataset\test"

In [2]:
import tensorflow
from tensorflow.keras.layers import Dense,Flatten,Input,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG, load_img
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers,models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SeparableConv2D,BatchNormalization,GlobalAveragePooling2D 
from tensorflow.keras.models import load_model
from PIL import Image
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np

In [ ]:
img_size=180
img_size_dim=[180,180]
dimension=(img_size,img_size)
zoom=[0.99,1.01]
bright=[0.8,1.2]
fill_mode='constant'
data_format='channels_last'
dir='Alzheimer_s Dataset\train'
data_gen=IDG(rescale=1./255,brightness_range=bright,zoom_range=zoom,data_format=data_format,fill_mode=fill_mode,horizontal_flip=True)
train_data_gen=data_gen.flow_from_directory(directory=dir,target_size=dimension,batch_size=6000,shuffle=False)

In [ ]:
train_data,train_labels=train_data_gen.next()
print(train_data.shape,train_labels.shape)

(5121, 180, 180, 3) (5121, 4)


In [ ]:
#handling imbalanced classes
from imblearn.over_sampling import SMOTE
sm=SMOTE(random_state=47)
train_data,train_labels=sm.fit_resample(train_data.reshape(-1,img_size*img_size*3),train_labels)
train_data=train_data.reshape(-1,img_size,img_size,3)
print(train_data.shape,train_labels.shape)

(10240, 180, 180, 3) (10240, 4)


In [ ]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_labels,test_labels = train_test_split(train_data,train_labels,test_size=0.2,random_state=47)
train_data,val_data,train_labels,val_labels = train_test_split(train_data,train_labels,test_size=0.2,random_state=47)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Create a Sequential model
model = models.Sequential()

# Add the first convolutional layer with 32 filters, a 3x3 kernel, and ReLU activation
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)))

# Add a max pooling layer with a 2x2 pool size
model.add(layers.MaxPooling2D((2, 2)))

# Add a flattening layer to convert the 3D feature maps to 1D feature vectors
model.add(layers.Flatten())

# Add a fully connected layer with 64 units and ReLU activation
model.add(layers.Dense(32, activation='relu'))

# Add the output layer with 10 units (assuming 10 classes) and softmax activation for classification
model.add(layers.Dense(4, activation='softmax'))

# Compile the model with the Adam optimizer and categorical cross-entropy loss function
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=mt)

# Print a summary of the model architecture
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 89, 89, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 43, 43, 64)        0         
 g2D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 118336)            0         
                                                                 
 dense_22 (Dense)            (None, 64)               

In [ ]:
mt=[tensorflow.keras.metrics.CategoricalAccuracy(name='Accuracy'),tensorflow.keras.metrics.AUC(name='AUC')]

In [ ]:
train_stat=model.fit(train_data,train_labels,validation_data=(val_data,val_labels),epochs=5)

Epoch 1/5
132/132 [==============================] - 51s 380ms/step - loss: 1.1607 - Accuracy: 0.5405 - AUC: 0.7985 - val_loss: 0.7170 - val_Accuracy: 0.6501 - val_AUC: 0.9048
Epoch 2/5
132/132 [==============================] - 50s 376ms/step - loss: 0.5293 - Accuracy: 0.7744 - AUC: 0.9507 - val_loss: 0.4600 - val_Accuracy: 0.8141 - val_AUC: 0.9629
Epoch 3/5
132/132 [==============================] - 48s 362ms/step - loss: 0.3661 - Accuracy: 0.8560 - AUC: 0.9764 - val_loss: 0.4032 - val_Accuracy: 0.8313 - val_AUC: 0.9700
Epoch 4/5
132/132 [==============================] - 48s 360ms/step - loss: 0.2091 - Accuracy: 0.9220 - AUC: 0.9924 - val_loss: 0.3342 - val_Accuracy: 0.8637 - val_AUC: 0.9793
Epoch 5/5
132/132 [==============================] - 47s 355ms/step - loss: 0.1539 - Accuracy: 0.9442 - AUC: 0.9956 - val_loss: 0.4511 - val_Accuracy: 0.8437 - val_AUC: 0.9671


In [ ]:
from keras.models import Model

def freeze_layers(model):
    for i in model.layers:
        i.trainable = False
        if isinstance(i, Model):
            freeze_layers(i)
    return model

In [ ]:
model_freezed = freeze_layers(model)
model_freezed.save('alzheimer_3.h5')

C:\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model_freezed.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 178, 178, 32)      896       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 89, 89, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 87, 87, 64)        18496     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 43, 43, 64)        0         
 g2D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 118336)            0         
                                                                 
 dense_22 (Dense)            (None, 64)               

In [ ]:
test_model=load_model("alzheimer_3.h5")

In [ ]:
img=image.load_img("Alzheimer_s Dataset\test\NonDemented\26 (62).jpg")
img=img.resize((180,180))

In [ ]:
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)

In [ ]:
print(np.argmax(test_model.predict(x),axis=1)[0])

1/1 [==============================] - 0s 28ms/step
2
